## 2.3. Setup Function Tools for ReAct Agent

In [3]:
from langchain_core.tools import tool

#Tool annotation identifies a function as a tool automatically
@tool
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

@tool
def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y


## 2.4. Create a basic ReAct Agent

In [4]:
from langchain_ollama import ChatOllama
# from langchain_openai import ChatOpenAI
# Setup the LLM for the agent

# Setup the LLM
model = ChatOllama(model="llama3.2")
# model = ChatOpenAI(model="gpt-5")

# Setup the GPT model

# Test the model
# response = model.invoke("Hello, how are you?")
# print(response.content)

In [5]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create list of tools available to the agent
agent_tools=[find_sum, find_product]

#System prompt
system_prompt = SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available.
    Do not solve the problem yourself"""
)

agent_graph=create_react_agent(
    model=model,
    prompt=system_prompt,
    tools=agent_tools)


## 2.5. Execute the ReAct Agent

In [6]:
#Example 1
inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : The sum of 2 and 3 is 5. 

Step by Step execution : 
================================ Human Message =================================

what is the sum of 2 and 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_sum (a5a11c6e-e04e-4a44-8c1a-79e5a0712ef9)
 Call ID: a5a11c6e-e04e-4a44-8c1a-79e5a0712ef9
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5
================================== Ai Message ==================================

The sum of 2 and 3 is 5.


In [9]:
from pprint import pprint

#Example 2
inputs = {"messages":[("user","What is 3 multipled by 2 and 5 + 1 ?")]}

result = agent_graph.invoke(inputs)
pprint(result)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

{'messages': [HumanMessage(content='What is 3 multipled by 2 and 5 + 1 ?', additional_kwargs={}, response_metadata={}, id='3054e863-6b5c-4db4-a1a3-9f605d0c3c3a'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-09-18T14:38:29.621937Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1244868334, 'load_duration': 67941875, 'prompt_eval_count': 309, 'prompt_eval_duration': 727727125, 'eval_count': 23, 'eval_duration': 446425541, 'model_name': 'llama3.2'}, id='run--b488ce0b-2518-4915-a9d4-4260a385f6c9-0', tool_calls=[{'name': 'find_product', 'args': {'x': '3', 'y': '7'}, 'id': '827c6854-983d-42c0-8678-0b46398d6afe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 309, 'output_tokens': 23, 'total_tokens': 332}),
              ToolMessage(content='21', name='find_product', id='a05a44cf-5a47-4381-8b9a-b10616f61831', tool_call_id='827c6854-983d-42c0-8678-0b46398d6afe'),
              AIMessage(content='The prod

## 2.6. Debugging the Agent

In [ ]:
agent_graph=create_react_agent(
    model=model,
    prompt=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)